In [1]:
import pandas as pd

# Step 1: 读取原始 CSV 数据
df = pd.read_csv("./LSTM-Multivariate_pollution.csv")

# Step 2: 解析时间列并排序
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").reset_index(drop=True)

# Step 3: 时间特征构造
df["hour"] = df["date"].dt.hour
df["weekday"] = df["date"].dt.weekday
df["is_weekend"] = df["weekday"].isin([5, 6]).astype(int)

# Step 4: 风向类别编码（转整数）
df["wnd_dir"] = df["wnd_dir"].astype("category").cat.codes

# Step 5: 构造滞后特征（历史污染）
for lag in [1, 3, 6, 24]:
    df[f"pollution_lag{lag}"] = df["pollution"].shift(lag)

# Step 6: 构造空气质量分类标签（污染级别 0~4）
def classify_aqi(x):
    if x <= 50:
        return 0  # 优
    elif x <= 100:
        return 1  # 良
    elif x <= 150:
        return 2  # 轻度污染
    elif x <= 200:
        return 3  # 中度污染
    else:
        return 4  # 重度污染

df["pollution_level"] = df["pollution"].apply(classify_aqi)

# Step 7: 删除因 shift 带来的缺失行
df = df.dropna().reset_index(drop=True)

# Step 8: 保存处理结果
df.to_csv("processed_pollution_multivariate.csv", index=False)
print("✅ 已保存为 processed_pollution_multivariate.csv")


✅ 已保存为 processed_pollution_multivariate.csv
